# Prior estimation

In [1]:
include("src/main.jl")
OPTCONFIG=OptConfig(XTOLABS=1e-8, DEBUG=true);

## The Model

In [2]:
# exponential
xlims = (0,4)
n = 200 # was 500
ndata = 100
σ = 0.3
prior = Truncated(MixtureModel([Normal(mu, .5) for mu in [1, 3]]), xlims...)


t    = Transformation(x->exp(x), x->log(x))

Transformation(#80, #81)

In [ ]:
for i = 1:100
  dir = "batch/$i"
  try
    mkdir(dir)
  end

  srand(i)
  m = FEModel(f=x->x, xs=linspace(xlims...,n), σ=σ)
  d = generatedata(m, prior, ndata);
  mt   = transformmodel(m, t)

  ncv = 20
  kfold = length(d);

  lims = ([0,4],[0,.6],[1,55], [0,.2])

  figsize=(4,3)

  ## er estimator
  regs = [ReferenceRegularizer(m, γ) for γ in linspace(0.1,2,ncv)]
  @time ss = [cvscore(m, d, d->ebprior(m, d, r), kfold) for r in regs]
  figure()
  PyPlot.plot([r.γ for r in regs], ss)
  PyPlot.savefig("$dir/ebcv.pdf")
  regR = regs[indmax(ss)].γ

  wR  = ebprior(m,  d, ReferenceRegularizer(m,  regR))
  wRt = ebprior(mt, d, ReferenceRegularizer(mt, regR))

  compareplot(m, t, pdf.(prior, m.xs), wR, wRt, "eb", lims...; figsize=figsize, savename="$dir/eb")

  ## thik estimator
  regs = [ThikonovRegularizer(γ) for γ in linspace(1,160,ncv)]
  @time ss = [cvscore(m, d, d->ebprior(m, d, r), kfold) for r in regs]
  figure()
  PyPlot.plot([r.γ for r in regs], ss)  
  PyPlot.savefig("$dir/l2cv.pdf")
  regT = regs[indmax(ss)].γ

  wT = ebprior(m,  d, ThikonovRegularizer(regT))
  wTt = ebprior(mt, d, ThikonovRegularizer(regT))

  compareplot(m, t, pdf.(prior, m.xs), wT, wTt, "L_2", lims...; figsize=figsize, savename="$dir/l2")

end

108.301456 seconds (165.90 M allocations: 109.587 GiB, 10.29% gc time)
112.805316 seconds (170.27 M allocations: 220.128 GiB, 17.02% gc time)
 51.578741 seconds (166.35 M allocations: 118.726 GiB, 18.49% gc time)
135.898199 seconds (170.29 M allocations: 221.170 GiB, 14.56% gc time)
 94.548037 seconds (165.41 M allocations: 100.725 GiB, 10.19% gc time)
 79.063284 seconds (169.87 M allocations: 211.910 GiB, 21.03% gc time)
 43.572222 seconds (165.54 M allocations: 103.117 GiB, 18.10% gc time)
 78.456150 seconds (169.69 M allocations: 208.041 GiB, 21.00% gc time)
 46.429552 seconds (166.28 M allocations: 117.398 GiB, 19.59% gc time)
 87.225828 seconds (171.43 M allocations: 246.121 GiB, 22.43% gc time)
 44.019558 seconds (165.94 M allocations: 110.801 GiB, 19.87% gc time)
 74.479569 seconds (169.78 M allocations: 209.930 GiB, 22.11% gc time)
 48.034329 seconds (166.57 M allocations: 122.810 GiB, 19.89% gc time)
 82.758575 seconds (171.34 M allocations: 244.062 GiB, 23.34% gc time)
 43.65

 90.874099 seconds (165.58 M allocations: 103.947 GiB, 10.30% gc time)
138.569099 seconds (169.88 M allocations: 212.160 GiB, 14.19% gc time)
 49.666970 seconds (165.97 M allocations: 111.423 GiB, 17.69% gc time)
